jupyter nbconvert 5_association.ipynb --to script

cd /data/ch6845/MHC*;screen -S assoc;

for i in {0..10};do python 4_association.py $i;done


5*3=15

for i in {00..101};do python 5_association.py $i;done

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6292650/

In [85]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink

import statsmodels.api as sm

from basic_tools import *

HLA_re_exp=re.compile('(?P<name>HLA_[0-9A-Z]*)\*(?P<allele>[0-9:]*)')
AA_re_exp=re.compile('(?P<name>AA_[A-Z0-9]*_[0-9]*_[0-9]*_exon[0-9]*)_*(?P<allele>[A-Z]*)')

plink_KCHIP_HLA_AA_SNP_1000G_path=data_genotype_merge_path+'KCHIP_HLA_AA_SNP_1000G'
plink_KCHIP_HLA_SNP_1000G_path=data_genotype_merge_path+'KCHIP_HLA_SNP_1000G' # for association
plink_1000G_path=data_genotype_merge_path+'1000G'

In [2]:
#plink_KCHIP_HLA_SNP_1000G=PyPlink(plink_KCHIP_HLA_SNP_1000G_path)
#plink_KCHIP_HLA_SNP_1000G_fam=plink_KCHIP_HLA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
#plink_KCHIP_HLA_SNP_1000G_bim=plink_KCHIP_HLA_SNP_1000G.get_bim()

In [3]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [4]:
plink_1000G=PyPlink(plink_1000G_path)
plink_1000G_fam=plink_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_1000G_bim=plink_1000G.get_bim()
# plink_KCHIP_HLA_SNP_1000G_bim -> for parameter

In [5]:
phased_KCHIP_HLA_AA_path
phased_KCHIP_HLA_AA_SNP_path

'data/genotype/4_merge/KCHIP_HLA_AA_SNP.bgl.phased'

# parse parameter

In [6]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t')
phenotypes=phenotypes.set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]

binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])
#binary_continuous_traits

if 'ipykernel' in sys.argv[0]:
    ipykernel=True
    phenotype_name='ALP'
    #phenotype_name='height'
else:
    ipykernel=False
    phenotype_name=sys.argv[1]
if phenotype_name.isdigit():
    phenotype_name=int(phenotype_name)
    phenotype_name=binary_continuous_traits[phenotype_name]      

In [7]:
pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
phenotype_type='binary' if len(pheno['pheno'].value_counts())<3 else 'continuous'

In [8]:
data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
pathlib.Path(data_out_assoc_phenotype_path).mkdir(parents=True, exist_ok=True)

In [9]:
f=open(phased_KCHIP_HLA_AA_path,'r');aa_ind=f.readline().strip().split(' ')[2:];f.close()

aa_ind_1=[aa_ind[i] for i in range(0,len(aa_ind),2)]
aa_ind_2=[aa_ind[i+1] for i in range(0,len(aa_ind),2)]

assert np.all(pheno['FID']==aa_ind_1)
assert np.all(pheno['FID']==aa_ind_2)

In [10]:
log = logging.getLogger('logger')
log.setLevel(logging.DEBUG)

log_file_name=datetime.datetime.now().strftime('%Y%m%d_%H%M%S')+'.log'
log_file_path=data_out_assoc_phenotype_path+log_file_name
fileHandler = logging.FileHandler(log_file_path)
streamHandler = logging.StreamHandler()

formatter = logging.Formatter('> %(message)s')
fileHandler.setFormatter(formatter)
streamHandler.setFormatter(formatter)

log.addHandler(fileHandler)
log.addHandler(streamHandler)

In [11]:
log.info_head=lambda x: log.info('-'*int((100-len(x))/2)+x+'-'*int((100-len(x))/2))

In [12]:
log.info_head("phenotype_name: {}, phenotype_type:{}".format(phenotype_name,phenotype_type))

> ---------------------------phenotype_name: ALP, phenotype_type:continuous---------------------------


# Association

plink_KCHIP_HLA_AA_SNP_1000G_HLAassign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[HLA_name]=0
    gene_select=plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[plink_KCHIP_HLA_AA_SNP_1000G_HLAassign.index.str.contains('HLA_'+HLA_name)|plink_KCHIP_HLA_AA_SNP_1000G_HLAassign.index.str.contains('SNPS_'+HLA_name)|plink_KCHIP_HLA_AA_SNP_1000G_HLAassign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[HLA_name][(plink_KCHIP_HLA_AA_SNP_1000G_HLAassign['pos']>=plink_KCHIP_HLA_AA_SNP_1000G_HLAassign['pos'].min())&(plink_KCHIP_HLA_AA_SNP_1000G_HLAassign['pos']<=plink_KCHIP_HLA_AA_SNP_1000G_HLAassign['pos'].max())]=1
plink_KCHIP_HLA_AA_SNP_1000G_HLAassign

In [13]:
#for i in gene_assign[gene_assign.A==1].index:
#    print(i,gene_assign.loc[i])

In [15]:
plink_KCHIP_HLA_AA_SNP_1000G_HLAassign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[HLA_name]=0
    gene_select=plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[plink_KCHIP_HLA_AA_SNP_1000G_HLAassign.index.str.contains('HLA_'+HLA_name)|plink_KCHIP_HLA_AA_SNP_1000G_HLAassign.index.str.contains('SNPS_'+HLA_name)|plink_KCHIP_HLA_AA_SNP_1000G_HLAassign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[HLA_name][(plink_KCHIP_HLA_AA_SNP_1000G_HLAassign['pos']>=gene_select['pos'].min())&(plink_KCHIP_HLA_AA_SNP_1000G_HLAassign['pos']<=gene_select['pos'].max())]=1
plink_KCHIP_HLA_AA_SNP_1000G_HLAassign

/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3319: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://p

,pos,A,B,C,DPA1,DPB1,DQA1,DQB1,DRB1
snp,,,,,,,,,
6:28000361_T/C,28000361,0,0,0,0,0,0,0,0
6:28000610_C/A,28000610,0,0,0,0,0,0,0,0
6:28001003_A/C,28001003,0,0,0,0,0,0,0,0
6:28001033_T/C,28001033,0,0,0,0,0,0,0,0
6:28001610_T/C,28001610,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
6:34997188_C/T,34997188,0,0,0,0,0,0,0,0
6:34997259_G/A,34997259,0,0,0,0,0,0,0,0
6:34997606_T/A,34997606,0,0,0,0,0,0,0,0


In [16]:
plink_KCHIP_HLA_AA_SNP_1000G_r2_list_dict={}

In [17]:
conditional_variant_list=['AA_A_-1','SNPS_A_28_30018337_exon1','6:30165273_C/T']

In [18]:
phased_KCHIP_HLA_AA_path

'data/genotype/4_merge/KCHIP_HLA_AA.bgl.phased'

In [ ]:
#Initialize *.aa
idx=1

In [ ]:
log.info_head("Step {}-1 Initialization".format(idx))

if os.path.exists('{}'.format(data_out_assoc_phenotype_path+'step_{:02d}.bgl.phased'.format(idx))):
    command='rm {}'.format(data_out_assoc_phenotype_path+'step_{:02d}.*'.format(idx))
    log.info(command);os.system(command)


command='cp {} {}'.format(phased_KCHIP_HLA_AA_path,data_out_assoc_phenotype_path+'step_{:02d}.bgl.phased'.format(idx))
log.info(command); assert os.system(command)==0

In [70]:
log.info_head("Step {}-2 Classifying conditional variant".format(idx))

conditional_variant_list_HLA_AA=[]
conditional_variant_list_phased_SNP=[]
conditional_variant_list_unphased_SNP=[]


for conditional_variant in conditional_variant_list:
    log.info("conditional_variant: {}".format(conditional_variant))
    if conditional_variant[:3]=='AA_':
        log.info("added to conditional_variant_list_HLA_AA")
        conditional_variant_list_HLA_AA.append(conditional_variant.split('_')[1])
    elif conditional_variant[:4]=='HLA_':
        log.info("added to conditional_variant_list_HLA_AA")
        conditional_variant_list_HLA_AA.append(conditional_variant[4:].split('*')[0])
    elif conditional_variant[:5]=='SNPS_':
        log.info("added to conditional_variant_list_HLA_AA")
        log.info("added to conditional_variant_list_phased_SNP")
        conditional_variant_list_HLA_AA.append(conditional_variant.split('_')[1])
        conditional_variant_list_phased_SNP.append(conditional_variant)
    else:
        if conditional_variant in plink_1000G_bim.index:
            log.info("added to conditional_variant_list_unphased_SNP")
            conditional_variant_list_unphased_SNP.append(conditional_variant)
        else:
            log.info("added to conditional_variant_list_phased_SNP")
            conditional_variant_list_phased_SNP.append(conditional_variant)
        log.info("calculating r2")
        r2_list=plink_KCHIP_HLA_AA_SNP_1000G_r2_list_dict.get(conditional_variant)
        if r2_list is None: 
            r2_list=[]
            for idx_bim,(SNP,row) in enumerate(plink_KCHIP_HLA_AA_SNP_1000G_bim.iterrows()):
                r2=pearsonr(plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(conditional_variant),plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(SNP))[0]**2
                r2_list.append(r2)
            plink_KCHIP_HLA_AA_SNP_1000G_r2_list_dict[conditional_variant]=r2_list
        r2_df=pd.DataFrame(r2_list,index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)             
        plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count=plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[r2_df[0]>0.7].sum(axis=0).drop('pos')
        log.info("r2 result:  \n{}".format(plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count))
        for HLA_name in plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count[plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count>0].index.tolist():
            log.info("{} added to conditional_variant_list_HLA_AA".format(HLA_name))
            conditional_variant_list_HLA_AA.append(HLA_name)


conditional_variant_list_HLA_AA=np.unique(conditional_variant_list_HLA_AA)
conditional_variant_list_phased_SNP=np.unique(conditional_variant_list_phased_SNP)
conditional_variant_list_unphased_SNP=np.unique(conditional_variant_list_unphased_SNP)


log.info("conditional_variant_list_HLA_AA {}".format(conditional_variant_list_HLA_AA))
log.info("conditional_variant_list_phased_SNP {}".format(conditional_variant_list_phased_SNP))
log.info("conditional_variant_list_unphased_SNP {}".format(conditional_variant_list_unphased_SNP))

> ------------------------------Step 1-2 Classifying conditional variant------------------------------
> conditional_variant: AA_A_-1
> added to conditional_variant_list_HLA_AA
> conditional_variant: SNPS_A_28_30018337_exon1
> added to conditional_variant_list_HLA_AA
> added to conditional_variant_list_phased_SNP
> conditional_variant: 6:30165273_C/T
> added to conditional_variant_list_unphased_SNP
> calculating r2
> r2 result:  
A       0
B       0
C       0
DPA1    0
DPB1    0
DQA1    0
DQB1    0
DRB1    0
dtype: int64
> conditional_variant_list_HLA_AA ['A']
> conditional_variant_list_phased_SNP ['SNPS_A_28_30018337_exon1']
> conditional_variant_list_unphased_SNP ['6:30165273_C/T']


In [71]:
print(1)

1


In [94]:
    log.info_head("Step {}-3 Constructing covariate and conditional list".format(idx))

    covariate_df=pd.read_csv(PC_path,sep='\t').set_index('ID').loc[plink_KCHIP_HLA_AA_SNP_1000G_fam['IID']]
    covariate_df['age']=phenotypes['age']
    covariate_df['sex']=phenotypes['sex']-1
    covariate_df['AS']=phenotypes['cohort'].replace(1,1).replace(2,0).replace(3,0)
    covariate_df['NC']=phenotypes['cohort'].replace(1,0).replace(2,0).replace(3,1)

    covariate_plink_df=covariate_df.copy()
    covariate_omnibus_df=covariate_df.copy()

    conditional_plink_list=[]
    conditional_omnibus_list=[]

    for HLA_gene in conditional_variant_list_HLA_AA:
        marker_dosage_list_dict={}
        for re_exp in [HLA_re_exp,AA_re_exp]:
            for marker in plink_KCHIP_HLA_AA_SNP_1000G_bim.index:
                name,allele=(re_exp.search(marker).group('name'),re_exp.search(marker).group('allele')) if re_exp.search(marker) is not None else (None,None)
                if name is not None and name.split('*')[0].split('_')[1]==HLA_gene:
                    marker_dosage_list=marker_dosage_list_dict.get(name,[])
                    marker_dosage_list.append({"marker":marker,"dosage":plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(marker)})
                    marker_dosage_list_dict[name]=marker_dosage_list

        for key,marker_dosage_list in marker_dosage_list_dict.items():
            marker_list=[marker_dosage['marker'] for marker_dosage in marker_dosage_list]
            dosage_array=np.array([marker_dosage['dosage'] for marker_dosage in marker_dosage_list]).transpose()
            if len(marker_list)>1:
                marker_list_cut=np.delete(marker_list,dosage_array.sum(axis=0).argmax())
                dosage_array_cut=np.delete(dosage_array,dosage_array.sum(axis=0).argmax(),axis=1)
            else:
                marker_list_cut=marker_list
                dosage_array_cut=dosage_array

            for i,marker in enumerate(marker_list_cut):
                covariate_plink_df[marker]=dosage_array_cut[:,i]

        conditional_omnibus_list+=marker_dosage_list_dict.keys()


    for phased_SNP in conditional_variant_list_phased_SNP:
        dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(phased_SNP).astype(float)
        dosage[dosage==-1]=np.nan

        covariate_plink_df[phased_SNP]=dosage
        #conditional_plink_list.append(phased_SNP)
        conditional_omnibus_list.append(phased_SNP)

        assert os.system('grep "{} " {} > {}temp.bgl.phased'.format(phased_SNP,phased_KCHIP_HLA_AA_SNP_path,data_out_assoc_phenotype_path))==0
        f=open('{}temp.bgl.phased'.format(data_out_assoc_phenotype_path),'r');lines=f.readlines();f.close();assert len(lines)==1
        assert os.system('cat {}temp.bgl.phased >> {}'.format(data_out_assoc_phenotype_path,data_out_assoc_phenotype_path+'step_{:02d}.bgl.phased'.format(idx)))==0
        assert os.system('rm {}temp.bgl.phased'.format(data_out_assoc_phenotype_path))==0    


    for unphased_SNP in conditional_variant_list_unphased_SNP:
        dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(unphased_SNP).astype(float)
        dosage[dosage==-1]=np.nan
        covariate_plink_df[unphased_SNP]=dosage
        covariate_omnibus_df[unphased_SNP]=dosage

    log.info("conditional_plink_list {} : {}".format(len(conditional_plink_list),conditional_plink_list))
    log.info("-----------")
    log.info("covariate_plink_df {} : {}".format(len(covariate_plink_df.columns),covariate_plink_df.columns))         
    log.info("-----------")
    log.info("conditional_omnibus_list {} : {}".format(len(conditional_omnibus_list),conditional_omnibus_list))
    log.info("-----------")
    log.info("covariate_omnibus_df {} : {}".format(len(covariate_omnibus_df.columns),covariate_omnibus_df.columns))    


> ------------------------Step 1-3 Constructing covariate and conditional list------------------------
> conditional_plink_list 0 : []
> -----------
> covariate_plink_df 104 : Index(['PC1', 'PC2', 'PC3', 'PC4', 'age', 'sex', 'AS', 'NC', 'HLA_A*01:01',
       'HLA_A*02:01',
       ...
       'AA_A_294_30020314_exon5', 'AA_A_297_30020323_exon5',
       'AA_A_298_30020326_exon5', 'AA_A_299_30020329_exon5',
       'AA_A_307_30020353_exon5', 'AA_A_311_30020365_exon5',
       'AA_A_321_30020837_exon6', 'AA_A_334_30021018_exon7',
       'SNPS_A_28_30018337_exon1', '6:30165273_C/T'],
      dtype='object', length=104)
> -----------
> conditional_omnibus_list 61 : ['HLA_A', 'AA_A_9_30018537_exon2', 'AA_A_17_30018561_exon2', 'AA_A_44_30018642_exon2', 'AA_A_56_30018678_exon2', 'AA_A_62_30018696_exon2', 'AA_A_63_30018699_exon2', 'AA_A_65_30018705_exon2', 'AA_A_66_30018708_exon2', 'AA_A_67_30018711_exon2', 'AA_A_70_30018720_exon2', 'AA_A_73_30018729_exon2', 'AA_A_74_30018732_exon2', 'AA_A_76_3001873

In [95]:
    def check_duplicate(array,axis=1,threshold=0.9,keep='last'):
        array=array.copy()
        if isinstance(array, np.ndarray):
            pass
        elif isinstance(array, pd.core.frame.DataFrame):
            array=array.values

        if axis==1:
            pass
        elif axis==0:
            array=array.transpose()
        else:
            raise NotImplementedError

        dupcol_check=np.repeat(False,array.shape[1])
        for i in range(array.shape[1]):
            for j in range(i+1,array.shape[1]):
                x_i=array[:,i]
                x_j=array[:,j]
                if np.all(x_i==x_j) or np.corrcoef(x_i,x_j)[0,1]>threshold:
                    if keep==False:
                        dupcol_check[i]=True
                        dupcol_check[j]=True
                    elif keep=='last':   
                        if (not dupcol_check[i]) and (not dupcol_check[j]):
                            dupcol_check[j]=True
                    elif keep=='loose_last':
                        dupcol_check[j]=True
                    else:
                        raise NotImplementedError
        return dupcol_check

In [96]:
    log.info_head("Step {}-4 Checking colinearity".format(idx))
    covariate_plink_df_drop=covariate_plink_df#.drop(columns=covariate_plink_df.columns[check_duplicate(covariate_plink_df,threshold=0.8)])
    covariate_omnibus_df_drop=covariate_omnibus_df#.drop(columns=covariate_omnibus_df.columns[check_duplicate(covariate_omnibus_df,threshold=0.9)])
    
    log.info("conditional_plink_list {} : {}".format(len(conditional_plink_list),conditional_plink_list))
    log.info("-----------")
    log.info("covariate_plink_df {} : {}".format(len(covariate_plink_df.columns),covariate_plink_df.columns))         
    log.info("-----------")
    log.info("conditional_omnibus_list {} : {}".format(len(conditional_omnibus_list),conditional_omnibus_list))
    log.info("-----------")
    log.info("covariate_omnibus_df {} : {}".format(len(covariate_omnibus_df.columns),covariate_omnibus_df.columns))    


> -----------------------------------Step 1-4 Checking colinearity-----------------------------------
> conditional_plink_list 0 : []
> -----------
> covariate_plink_df 104 : Index(['PC1', 'PC2', 'PC3', 'PC4', 'age', 'sex', 'AS', 'NC', 'HLA_A*01:01',
       'HLA_A*02:01',
       ...
       'AA_A_294_30020314_exon5', 'AA_A_297_30020323_exon5',
       'AA_A_298_30020326_exon5', 'AA_A_299_30020329_exon5',
       'AA_A_307_30020353_exon5', 'AA_A_311_30020365_exon5',
       'AA_A_321_30020837_exon6', 'AA_A_334_30021018_exon7',
       'SNPS_A_28_30018337_exon1', '6:30165273_C/T'],
      dtype='object', length=104)
> -----------
> conditional_omnibus_list 61 : ['HLA_A', 'AA_A_9_30018537_exon2', 'AA_A_17_30018561_exon2', 'AA_A_44_30018642_exon2', 'AA_A_56_30018678_exon2', 'AA_A_62_30018696_exon2', 'AA_A_63_30018699_exon2', 'AA_A_65_30018705_exon2', 'AA_A_66_30018708_exon2', 'AA_A_67_30018711_exon2', 'AA_A_70_30018720_exon2', 'AA_A_73_30018729_exon2', 'AA_A_74_30018732_exon2', 'AA_A_76_30018738

In [97]:
    log.info_head("Step {}-5 Save covariate and conditional list".format(idx))
        
    pd.Series(conditional_plink_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)
    pd.Series(conditional_omnibus_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx),index=None,sep='\t',header=False)
    
    plink_KCHIP_HLA_AA_SNP_1000G_fam.iloc[:,:2].merge(right=covariate_plink_df_drop,left_on='IID',right_index=True).fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')
    plink_KCHIP_HLA_AA_SNP_1000G_fam.iloc[:,:2].merge(right=covariate_omnibus_df_drop,left_on='IID',right_index=True).fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar'.format(idx),index=None,sep='\t')


> ----------------------------Step 1-5 Save covariate and conditional list----------------------------


In [ ]:
            conditional_variant_list_phased_SNP.append(conditional_variant)
        log.info("calculating r2")
        r2_list=plink_KCHIP_HLA_AA_SNP_1000G_r2_list_dict.get(conditional_variant)
        if r2_list is None: 
            r2_list=[]
            for idx_bim,(SNP,row) in enumerate(plink_KCHIP_HLA_AA_SNP_1000G_bim.iterrows()):
                r2=pearsonr(plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(conditional_variant),plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(SNP))[0]**2
                r2_list.append(r2)
            plink_KCHIP_HLA_AA_SNP_1000G_r2_list_dict[conditional_variant]=r2_list
        r2_df=pd.DataFrame(r2_list,index=plink_KCHIP_HLA_AA_SNP_1000G_bim.index)             
        plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count=plink_KCHIP_HLA_AA_SNP_1000G_HLAassign[r2_df[0]>0.7].sum(axis=0).drop('pos')
        log.info("r2 result:  \n{}".format(plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count))
        for HLA_name in plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count[plink_KCHIP_HLA_AA_SNP_1000G_HLAassign_count>0].index.tolist():
            log.info("{} added to conditional_variant_list_HLA_AA".format(HLA_name))
            conditional_variant_list_HLA_AA.append(HLA_name)


In [151]:
model_result.nobs,model_result.params.iloc[-1],model_result.bse.iloc[-1],model_result.tvalues.iloc[-1],model_result.pvalues.iloc[-1]#.summary2()

(59468,
 0.05190567456517137,
 0.755766030830075,
 0.06867955484604434,
 0.9452446926391793)

In [150]:
for idx_bim,(SNP,row) in enumerate(plink_KCHIP_HLA_AA_SNP_1000G_bim.iterrows()):
    y_data=pheno['pheno'].replace(-9,np.nan)
    
    
    x_data_intercept=np.array([np.ones(plink_KCHIP_HLA_AA_SNP_1000G_fam.shape[0])]).transpose()    
    x_data_dosage=plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker(SNP).astype(float);x_data_dosage[x_data_dosage==-1]=np.nan
    x_data_dosage=np.expand_dims(x_data_dosage,axis=1)
    x_data_covariate=covariate_plink_df.values
    #print(x_data_intercept.shape,covariate_plink_df.values.shape)
    x_data=np.concatenate([x_data_intercept,x_data_covariate,x_data_dosage],axis=1)#[~x_y_data_nan]
    x_data_null=np.concatenate([x_data_intercept,x_data_covariate],axis=1)#[~x_y_data_nan]
    
    family=(sm.families.Gaussian() if phenotype_type=='continuous' else sm.families.Binomial())
    model=sm.GLM(y_data,x_data, family=family,missing='drop')
    model_null=sm.GLM(y_data,x_data_null, family=family,missing='drop')
    model_result=model.fit()
    model_result_null=model_null.fit()
    
    print(model_result.summary())
    if idx_bim==2:
        break
    


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  pheno   No. Observations:                59468
Model:                            GLM   Df Residuals:                    59425
Model Family:                Gaussian   Df Model:                           42
Link Function:               identity   Scale:                          5288.2
Method:                          IRLS   Log-Likelihood:            -3.3928e+05
Date:                Sun, 01 Mar 2020   Deviance:                   3.1425e+08
Time:                        22:46:35   Pearson chi2:                 3.14e+08
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.0973      4.012      4.013      0.0

In [115]:
x_data_dosage,np.expand_dims(x_data_dosage,axis=1)

(array([1., 0., 0., ..., 0., 0., 0.]), array([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]]))

In [ ]:
    x_data_null=np.concatenate([x_data_intercept,x_data_covar,x_data_condition,x_data_aa_marker],axis=1)[~x_y_data_nan]
    x_data_alt=np.concatenate([x_data_intercept,x_data_covar,x_data_condition],axis=1)[~x_y_data_nan]
    y_data=y_data[~x_y_data_nan]
    
    
    family=(sm.families.Gaussian() if assoc=='linear' else sm.families.Binomial())
    model_null = sm.GLM(y_data,x_data_null, family=family,missing='raise')
    model_alt = sm.GLM(y_data,x_data_alt, family=family,missing='raise')
    

In [84]:
    log.info_head("Step {}-6 Plink Association".format(idx))


    command='plink2 --bfile {bfile} {assoc_mode} --pheno {pheno} {covar_on} {covar} --condition-list {condition_list} --out {out} --covar-variance-standardize --threads 40'.format(bfile=plink_KCHIP_HLA_SNP_1000G_path,
                                                                                                                        assoc_mode='--logistic' if phenotype_type=='binary' else '--linear',
                                                                                                                         pheno=data_out_pheno_path+phenotype_name+'.phe',
                                                                                                                         covar_on='--covar' if len(covariate_plink_df.columns)!=0 else '',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx) if len(covariate_plink_df.columns)!=0 else '',
                                                                                                                         condition_list=data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),
                                                                                                                         out=data_out_assoc_phenotype_path+'step_{:02d}.plink'.format(idx)                                                                         
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False,quiet=True)
    log.info(stdout)
    log.error(stderr)

> -------------------------------------Step 1-6 Plink Association-------------------------------------
> plink2 --bfile data/genotype/4_merge/KCHIP_HLA_SNP_1000G --linear --pheno data/out_pheno/ALP.phe --covar data/out_assoc/ALP/step_01.plink.covar --condition-list data/out_assoc/ALP/step_01.plink.cond --out data/out_assoc/ALP/step_01.plink --covar-variance-standardize --threads 40


------RUN-----
plink2 --bfile data/genotype/4_merge/KCHIP_HLA_SNP_1000G --linear --pheno data/out_pheno/ALP.phe --covar data/out_assoc/ALP/step_01.plink.covar --condition-list data/out_assoc/ALP/step_01.plink.cond --out data/out_assoc/ALP/step_01.plink --covar-variance-standardize --threads 40


> PLINK v2.00a2.3LM 64-bit Intel (24 Jan 2020)   www.cog-genomics.org/plink/2.0/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/out_assoc/ALP/step_01.plink.log.
Options in effect:
  --bfile data/genotype/4_merge/KCHIP_HLA_SNP_1000G
  --condition-list data/out_assoc/ALP/step_01.plink.cond
  --covar data/out_assoc/ALP/step_01.plink.covar
  --covar-variance-standardize
  --glm
  --out data/out_assoc/ALP/step_01.plink
  --pheno data/out_pheno/ALP.phe
  --threads 40

Start time: Sun Mar  1 22:22:24 2020
257299 MiB RAM detected; reserving 128649 MiB for main workspace.
Using up to 40 threads (change this with --threads).
125673 samples (80026 females, 45647 males; 125673 founders) loaded from
data/genotype/4_merge/KCHIP_HLA_SNP_1000G.fam.
74320 variants loaded from data/genotype/4_merge/KCHIP_HLA_SNP_1000G.bim.
2 phenotypes loaded (1 binary, 1 quantitative).
40 covariates loaded from data/out_assoc/ALP/step_01.plink.covar.
--covar-variance-stan

In [ ]:
    log.info_head("Step {}-6 Plink Association".format(idx))


    command='plink2 --bfile {bfile} {assoc_mode} --pheno {pheno} {covar_on} {covar} --condition-list {condition_list} --out {out} --covar-variance-standardize --threads 40'.format(bfile=plink_KCHIP_HLA_SNP_1000G_path,
                                                                                                                        assoc_mode='--logistic' if phenotype_type=='binary' else '--linear',
                                                                                                                         pheno=data_out_pheno_path+phenotype_name+'.phe',
                                                                                                                         covar_on='--covar' if len(covariate_plink_df.columns)!=0 else '',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx) if len(covariate_plink_df.columns)!=0 else '',
                                                                                                                         condition_list=data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),
                                                                                                                         out=data_out_assoc_phenotype_path+'step_{:02d}.plink'.format(idx)                                                                         
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False,quiet=True)
    log.info(stdout)
    log.error(stderr)

In [58]:
plink_KCHIP_HLA_AA_SNP_1000G_fam.iloc[:,:2].merge(right=covariate_plink_df,left_on='IID',right_index=True).fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')
plink_KCHIP_HLA_AA_SNP_1000G_fam.iloc[:,:2].merge(right=covariate_omnibus_df,left_on='IID',right_index=True).fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar'.format(idx),index=None,sep='\t')


In [ ]:
    covariate_plink_df=covariate_plink_df.reset_index()
    covariate_plink=covariate_plink.drop(columns=covariate_plink.iloc[:,1:].columns[get_dupbool(covariate_plink.iloc[:,1:])])
    if covariate_plink.shape[1]>1:
        covariate_plink[[covariate_plink.columns[0],covariate_plink.columns[0]]+covariate_plink.columns[1:].values.tolist()].fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')
        #cova

In [ ]:
    log.info_head("Step {}-6 Save covariate and conditional list".format(idx))
        
    pd.Series(conditional_plink_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)
    #pd.Series(conditional_omnibus_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx))
    
    pd.Series(conditional_omnibus_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx),index=None,sep='\t',header=False)
    

In [ ]:
    #Run plink association test
    log.info("#########################################  STEP {}-2 Plink Association  #########################################".format(idx))
    command='plink2 --bfile {bfile} {assoc_mode} --pheno {pheno} {covar_on} {covar} --condition-list {condition_list} --out {out} --covar-variance-standardize --threads 40'.format(bfile=plink_path,
                                                                                                                        assoc_mode='--logistic' if phenotype_type=='binary' else '--linear',
                                                                                                                         pheno=data_out_pheno_path+phenotype_name+'.phe',
                                                                                                                         covar_on='--covar' if len(covariate_plink.columns[1:])!=0 else '',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx) if len(covariate_plink.columns[1:])!=0 else '',
                                                                                                                         condition_list=data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),
                                                                                                                         out=data_out_assoc_phenotype_path+'step_{:02d}.plink'.format(idx)                                                                         
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)
    

    
    log.info("#########################################  STEP {}-3 Omnibus Association  #########################################".format(idx))
    #Run omnibus association test
    command='python omnibus_test.py --assoc {assoc_mode} --fam {fam} --aa {aa} --pheno {pheno} {covar_on} {covar} --condition-list {condition_list} --out {out}'.format(assoc_mode='logistic' if phenotype_type=='binary' else 'linear',
                                                                                                                         aa=data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx),
                                                                                                                         fam=plink_path+'.fam',
                                                                                                                         pheno=data_out_pheno_path+phenotype_name+'.phe',
                                                                                                                         covar_on='--covar' if len(covariate_omnibus.columns[1:])!=0 else '',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar'.format(idx) if len(covariate_omnibus.columns[1:])!=0 else '',
                                                                                                                         condition_list=data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx),
                                                                                                                         out=data_out_assoc_phenotype_path+'step_{:02d}.omnibus'.format(idx)
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)    
    
    """
    log.info("#########################################  STEP {}-3 Omnibus Association  #########################################".format(idx))
    #Run omnibus association test
    command='Rscript OmnibusTest_BHv5_modified.R {assoc_mode} {output} {fam} {aa} {phe} {covar} {covarname} {condvar}'.format(assoc_mode='logistic' if phenotype_type=='binary' else 'linear',
                                                                                                                         output=data_out_assoc_phenotype_path+'step_{:02d}.omnibus'.format(idx),
                                                                                                                         aa=data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx),
                                                                                                                         fam=plink_path+'.fam',
                                                                                                                         phe=data_out_assoc_phenotype_path+'phenotype.phe',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar.old'.format(idx) if len(covariate_omnibus.columns[1:])!=0 else 'NA',
                                                                                                                         covarname=','.join(covariate_omnibus_old.columns[1:]) if len(covariate_omnibus_old.columns[1:])!=0 else 'NA',
                                                                                                                         condvar=','.join(conditional_omnibus_list+conditional_plink_phased_list) if len(conditional_omnibus_list+conditional_plink_phased_list)!=0 else 'NA'
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)
    
    """
    
    log.info("#########################################  STEP {}-4 Result Save  #########################################".format(idx))
    
    #Data save
    #plink_assoc_result=parse_plink_assoc(data_out_assoc_phenotype_path+'step_{:02d}.plink.{}assoc'.format(idx,'' if phenotype_type=='binary' else 'q'))
    #plink_assoc_result.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.{}assoc.munged'.format(idx,'' if phenotype_type=='binary' else 'q'))
    #plink_assoc_result_sorted=plink_assoc_result.sort_values('P')    
    #plink_assoc_result_sorted.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.{}assoc.sorted_'.format(idx,'' if phenotype_type=='binary' else 'q'),sep='\t',index=None)    
    
    plink_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t').astype({'P':float})
    plink_assoc_result=plink_assoc_result[plink_assoc_result['TEST']=='ADD']
    plink_assoc_result.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}.munged'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    plink_assoc_result_sorted=plink_assoc_result.sort_values('P')    
    plink_assoc_result_sorted.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}.sorted'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    
    omnibus_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.{}'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t').astype({'P':float})
    omnibus_assoc_result.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.{}.munged'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    omnibus_assoc_result_sorted=omnibus_assoc_result.sort_values('P')
    omnibus_assoc_result_sorted.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.{}.sorted'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    
    #Identify Peak variant
    log.info("Plink association result")
    log.info(plink_assoc_result_sorted.head(5))
    log.info("Omnibus association result")
    log.info(omnibus_assoc_result_sorted.head(5))
    
    if plink_assoc_result_sorted.iloc[0]['P'] < omnibus_assoc_result_sorted.iloc[0]['P']:
        variant_name=plink_assoc_result_sorted.iloc[0]['ID']
        log.info('Plink win!!! {}'.format(variant_name))
        conditional_variant_list.append(("plink",variant_name))        
        if plink_assoc_result_sorted.iloc[0]['P']>5e-8:
            log.info('p-value of {}({}) is larger than 5x10^-8. finish conditional analysis'.format(variant_name,plink_assoc_result_sorted.iloc[0]['P']))
            break
    else:
        variant_name=omnibus_assoc_result_sorted.iloc[0]['ID']
        log.info('Omnibus win!!! {}'.format(variant_name))
        conditional_variant_list.append(("omnibus",variant_name))
        if omnibus_assoc_result_sorted.iloc[0]['P']>5e-8:
            log.info('p-value of {}({}) is larger than 5x10^-8. finish conditional analysis'.format(variant_name,omnibus_assoc_result_sorted.iloc[0]['P']))
            break

In [90]:
np.unique(y_data)

array([ 4., 11., 12., ..., nan, nan, nan])

In [ ]:
for 

In [ ]:
covariate_plink_df.columns[check_duplicate(covariate_plink_df,threshold=0.9999,keep=False)]

In [ ]:
covariate_plink_df.columns[check_duplicate(covariate_plink_df,threshold=0.9999,keep=False)]

In [320]:
covariate_plink_df.columns[~check_duplicate(covariate_plink_df,threshold=0.9,keep=False)]

Index(['age', 'sex', 'AS', 'NC', 'HLA_A*02:01', 'HLA_A*02:03', 'HLA_A*02:06',
       'HLA_A*26:01', 'HLA_A*26:03', 'HLA_A*30:04', 'HLA_A*31:01',
       'HLA_A*33:03', 'AA_A_9_30018537_exon2_F', 'AA_A_9_30018537_exon2_Y',
       'AA_A_56_30018678_exon2', 'AA_A_62_30018696_exon2_Q',
       'AA_A_70_30018720_exon2', 'AA_A_76_30018738_exon2_A',
       'AA_A_77_30018741_exon2_N', 'AA_A_114_30019093_exon3_R',
       'AA_A_116_30019099_exon3_D', 'AA_A_152_30019207_exon3_E',
       'AA_A_156_30019219_exon3_Q', 'SNPS_A_28_30018337_exon1',
       '6:30165273_C/T'],
      dtype='object')

In [ ]:
log.info_head("Step {}-5 Save".format(idx))

In [ ]:
log.info_head("Step {}-6 Plink".format(idx))

In [ ]:
log.info_head("Step {}-7 Omnibus".format(idx))

In [ ]:
def check_duplicate(array,axis=1,threshold=0.9,keep='last'):
    array=array.copy()
    if isinstance(array, np.ndarray):
        pass
    elif isinstance(array, pd.core.frame.DataFrame):
        array=array.values
        
    if axis==1:
        pass
    elif axis==0:
        array=array.transpose()
    else:
        raise NotImplementedError
            
    dupcol_check=np.repeat(False,array.shape[1])
    for i in range(array.shape[1]):
        for j in range(i+1,array.shape[1]):
            x_i=array[:,i]
            x_j=array[:,j]
            if np.all(x_i==x_j) or np.corrcoef(x_i,x_j)[0,1]>threshold:
                if keep==False:
                    dupcol_check[i]=True
                    dupcol_check[j]=True
                elif keep=='last':   
                    if (not dupcol_check[i]) and (not dupcol_check[j]):
                        dupcol_check[j]=True
                elif keep=='loose_last':
                    dupcol_check[j]=True
                else:
                    raise NotImplementedError
    return dupcol_check

In [311]:
check_duplicate(covariate_plink_df,keep==False)

Signature: check_duplicate(array, axis=1, threshold=0.9, keep='last')
Docstring: <no docstring>
File:      /data/ch6845/MHC_phewas_testbench/<ipython-input-307-1ff9a2e80d4c>
Type:      function


In [ ]:
AA_re_exp=re.compile('AA_(?P<name>[0-9A-Z]*)_(?P<allele>[0-9]*)_.*')
for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('AA_',regex=True)].index:
    print(i)

In [ ]:
np.delete(dosage_array,dosage_array.sum(axis=0).argmax(),axis=1)

In [ ]:
conditional_variant_list=[('plink', '6:30165273_C/T'), ('plink', '6:30170628_T/TA'), ('plink', 'rs2523942'), ('plink', '6:29917446_T/G')]
r2_list_dict={}

for idx in range(5,10+1):
    log.info("############################################  STEP {}  ############################################".format(idx))
    log.info("##########################  STEP {}-1 Prepare input (covar,cond,etc.)  ###################################".format(idx))
    #Initialize *.aa
    if os.path.exists('{}'.format(data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx))):
        command='rm {}'.format(data_out_assoc_phenotype_path+'step_{:02d}*'.format(idx))
        log.info(command);os.system(command)
    
    #command='cp {} {}'.format()
    #print(command);os.system(command)
    
    command='head -n5 {} > {}'.format(aa_path,data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx))
    log.info(command); assert os.system(command)==0
    
    command='grep " AA_" {} >> {}'.format(aa_path,data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx))
    log.info(command); assert os.system(command)==0

    
    #Initialize conditional
    conditional_omnibus_list=[]
    conditional_plink_list=[]
    conditional_plink_phased_list=[]
    
    #Initialize covariate
    #covariate_plink=pd.DataFrame(index=phenotypes.index) 
    #covariate_omnibus=pd.DataFrame(index=phenotypes.index) 
    
    covariate_plink=pd.read_csv(PC_path,sep='\t').set_index('ID').loc[fam['IID']]
    covariate_plink['age']=phenotypes['age']
    covariate_plink['sex']=phenotypes['sex']
    covariate_plink['AS']=phenotypes['cohort'].replace(1,1).replace(2,0).replace(3,0)
    covariate_plink['CT']=phenotypes['cohort'].replace(1,0).replace(2,1).replace(3,0)
    #covariate_plink['NC']=phenotypes['cohort'].replace(1,0).replace(2,0).replace(3,1)
    
    covariate_omnibus=pd.read_csv(PC_path,sep='\t').set_index('ID').loc[fam['IID']]
    covariate_omnibus['age']=phenotypes['age']
    covariate_omnibus['sex']=phenotypes['sex']
    covariate_omnibus['AS']=phenotypes['cohort'].replace(1,1).replace(2,0).replace(3,0)
    covariate_omnibus['CT']=phenotypes['cohort'].replace(1,0).replace(2,1).replace(3,0)
    #covariate_omnibus['NC']=phenotypes['cohort'].replace(1,0).replace(2,0).replace(3,1)    
    
    log.info("##########################  STEP {}-1-1 conditional variant expansion (assign to a HLA gene)  ###################################".format(idx))
    log.info("conditional_variant_list:"+str(conditional_variant_list))
    conditional_variant_list_expand=[]
    #expand
    for assoc_type,conditional_variant in conditional_variant_list:
        log.info('expanding {}'.format(conditional_variant))
        if assoc_type=='omnibus':
            for variant_name in np.unique(['_'.join(variant.split('_')[:3]) for variant in bim_aa[bim_aa.index.str.contains('AA_'+conditional_variant.split('_')[1]+'_')].index.tolist()]):#AA
                conditional_variant_list_expand.append(("omnibus",variant_name))
                log.info('found {}'.format(variant_name))
            for variant_name in bim_aa[bim_aa.index.str.contains('HLA_'+conditional_variant.split('_')[1]+'\*')].index:#HLA
                conditional_variant_list_expand.append(("plink",variant_name))
                log.info('found {}'.format(variant_name))
        elif assoc_type=='plink':
            if conditional_variant[:3]=='HLA':
                for variant_name in np.unique(['_'.join(variant.split('_')[:3]) for variant in bim_aa[bim_aa.index.str.contains('AA_'+conditional_variant[4:].split('*')[0]+'_')].index.tolist()]):#AA
                    conditional_variant_list_expand.append(("omnibus",variant_name))
                    log.info('found {}'.format(variant_name))
                for variant_name in bim_aa[bim_aa.index.str.contains('HLA_'+conditional_variant[4:].split('*')[0]+'\*')].index:#HLA
                    conditional_variant_list_expand.append(("plink",variant_name))                
                    log.info('found {}'.format(variant_name))
            else:#SNPs
                conditional_variant_list_expand.append(("plink",conditional_variant))
                log.info('found {}'.format(conditional_variant))
                
                r2_list=r2_list_dict.get(conditional_variant)
                if r2_list is None: 
                    r2_list=[]
                    for idx_bim,(SNP,row) in enumerate(bim_aa.iterrows()):
                        r2=pearsonr(plink_aa.get_geno_marker(conditional_variant),plink_aa.get_geno_marker(SNP))[0]**2
                        r2_list.append(r2)
                    r2_list_dict[conditional_variant]=r2_list
                r2_df=pd.DataFrame(r2_list,index=bim_aa.index)    

                gene_assign_count=gene_assign[r2_df[0]>0.7].sum(axis=0).iloc[1:]    
                log.info("found HLA gene with LD \n "+str(gene_assign_count))
                
                for HLA_type in gene_assign_count[gene_assign_count>0].index.tolist():
                    for variant_name in np.unique(['_'.join(variant.split('_')[:3]) for variant in bim_aa[bim_aa.index.str.contains('AA_'+HLA_type+'_')].index.tolist()]):#AA
                        conditional_variant_list_expand.append(("omnibus",variant_name))
                        log.info('found {}'.format(variant_name))
                    for variant_name in bim_aa[bim_aa.index.str.contains('HLA_'+HLA_type+'\*')].index:#HLA
                        conditional_variant_list_expand.append(("plink",variant_name))
                        log.info('found {}'.format(variant_name))
        else:
            raise
    log.info("conditional_variant_list_expand: "+str(conditional_variant_list_expand))
    log.info("##########################  STEP {}-1-2 conditional variant -> omnibus,plink,plink_phased  ###################################".format(idx))
    #Split conditional
    for assoc_type,conditional_variant in conditional_variant_list_expand:
        if assoc_type=='omnibus':
            conditional_omnibus_list.append(conditional_variant)
            log.info("{} -> omnibus".format(conditional_variant))
        elif assoc_type=='plink':
            if os.path.exists('{}temp.aa'.format(data_out_assoc_phenotype_path)):
                os.system('rm {}temp.aa'.format(data_out_assoc_phenotype_path))
            os.system('grep "{} " {} > {}temp.aa'.format(conditional_variant.replace('*','\*'),aa_path,data_out_assoc_phenotype_path));f=open('{}temp.aa'.format(data_out_assoc_phenotype_path),'r');lines=f.readlines();f.close();os.system('rm {}temp.aa'.format(data_out_assoc_phenotype_path))
            if len(lines)==0:
                conditional_plink_list.append(conditional_variant)
                log.info("{} -> plink".format(conditional_variant))
            elif len(lines)==1:
                conditional_plink_phased_list.append(conditional_variant)
                log.info("{} -> plink_phased".format(conditional_variant))
            else:
                raise
        else:
            raise

    conditional_omnibus_list=list(np.unique(conditional_omnibus_list))
    conditional_plink_list=list(np.unique(conditional_plink_list))
    conditional_plink_phased_list=list(np.unique(conditional_plink_phased_list))
    
    log.info('conditional_omnibus_list: '+str(conditional_omnibus_list))
    log.info('conditional_plink_list: '+str(conditional_plink_list))
    log.info('conditional_plink_phased_list: '+str(conditional_plink_phased_list))
    
    
    #Process condition
    #(1) Omnibus AA peak
    log.info("##########################  STEP {}-1-3 process conditional_omnibus_list  ###################################".format(idx))
    
    
    for conditional_variant in conditional_omnibus_list:
        log.info("Process {}".format(conditional_variant))
        #individual_list=fam.IID#pd.read_csv(plink_path+'.fam',header=None,sep=' ')[1]
        if os.path.exists('{}temp.aa'.format(data_out_assoc_phenotype_path)):
            os.system('rm {}temp.aa'.format(data_out_assoc_phenotype_path))
        os.system('grep "{} " {} > {}temp.aa'.format(conditional_variant.replace('*','\*'),aa_path,data_out_assoc_phenotype_path));f=open('{}temp.aa'.format(data_out_assoc_phenotype_path),'r');lines=f.readlines();f.close();os.system('rm {}temp.aa'.format(data_out_assoc_phenotype_path))
        marker_name=lines[0].split(' ')[1]
        assert marker_name==conditional_variant;assert len(lines)==1
        aa_call_list=np.array(lines[0].strip().split(' ')[2:])
        aa_call_list_group=np.array([(aa_call_list[i],aa_call_list[i+1]) for i in range(0,len(aa_call_list),2)])

        residue_dosage_all=pd.DataFrame(index=fam.IID)
        for residue in np.unique(aa_call_list):
            log.info("residue {} ".format(residue)+("skipped" if residue=='NA' else "dosage count"))
            if residue=="NA":
                continue
            residue_dosage=pd.Series(0,index=fam.IID)
            residue_dosage.loc[(aa_call_list_group[:,0]==residue)]+=1
            residue_dosage.loc[(aa_call_list_group[:,1]==residue)]+=1
            residue_dosage_all[residue]=residue_dosage
        log.info("ignored {} among \n{}".format(residue_dosage_all.sum().sort_values(ascending=True).index[-1],residue_dosage_all.sum().sort_values(ascending=True)))

        residue_dosage_all_excluded=residue_dosage_all[residue_dosage_all.sum().sort_values(ascending=True)[:-1].index]#exclude the most frequent residue
        for residue,row in residue_dosage_all_excluded.items():
            covariate_plink[conditional_variant+'_'+residue]=row        
        log.info("plink covariate "+str(covariate_plink.columns))    
        
    #(2) Plink unphased peak (1000G-SNP)
    log.info("##########################  STEP {}-1-4 process conditional_plink_list  ###################################".format(idx))
    for conditional_variant in conditional_plink_list:
        log.info("** process {}".format(conditional_variant))
        dosage=plink.get_geno_marker(conditional_variant).astype(float)
        dosage[dosage==-1]=np.nan
        covariate_omnibus[conditional_variant]=dosage
        log.info("added dosage of {} to omnibus covariate".format(conditional_variant))
        
    #(3) Plink phased peak (KCHIP-HLA, SNP)
    log.info("##########################  STEP {}-1-5 process conditional_plink_phased_list  ###################################".format(idx))
    for conditional_variant in conditional_plink_phased_list:
        log.info("** process {}".format(conditional_variant))
        #Attach to aa file
        if os.path.exists('{}temp.aa'.format(data_out_assoc_phenotype_path)):
            os.system('rm {}temp.aa'.format(data_out_assoc_phenotype_path));
        
        os.system('grep "{} " {} > {}temp.aa'.format(conditional_variant.replace('*','\*'),aa_path,data_out_assoc_phenotype_path));f=open('{}temp.aa'.format(data_out_assoc_phenotype_path),'r');lines=f.readlines();f.close();assert len(lines)==1
        os.system('cat {}temp.aa >> {}'.format(data_out_assoc_phenotype_path,data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx)))
        os.system('rm {}temp.aa'.format(data_out_assoc_phenotype_path))
        log.info("added phased information of {} to aa file".format(conditional_variant))
    
    
    
    log.info("##########################  STEP {}-1-6 save covariate, condition  ###################################".format(idx))
    log.info('convarite_omnibus: '+str(covariate_omnibus.columns[:] if len(covariate_omnibus.columns)!=0 else []))
    log.info('convarite_plink: '+str(covariate_plink.columns[:] if len(covariate_plink.columns)!=0 else []))
    
    
    
    covariate_plink=covariate_plink.reset_index()
    covariate_plink=covariate_plink.drop(columns=covariate_plink.iloc[:,1:].columns[get_dupbool(covariate_plink.iloc[:,1:])])
    if covariate_plink.shape[1]>1:
        covariate_plink[[covariate_plink.columns[0],covariate_plink.columns[0]]+covariate_plink.columns[1:].values.tolist()].fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')
        #covariate_plink[[covariate_plink.columns[0],covariate_plink.columns[0]]].to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')
    
    covariate_omnibus=covariate_omnibus.reset_index()
    covariate_omnibus=covariate_omnibus.drop(columns=covariate_omnibus.iloc[:,1:].columns[get_dupbool(covariate_omnibus.iloc[:,1:])])
    if covariate_omnibus.shape[1]>1:
        covariate_omnibus[[covariate_omnibus.columns[0],covariate_omnibus.columns[0]]+covariate_omnibus.columns[1:].values.tolist()].fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar'.format(idx),index=None,sep='\t')
        
    """
    ##FOR OLD
    covariate_omnibus_old=covariate_omnibus.copy()
    covariate_omnibus_old.columns=['header{}'.format(i) for i in range(len(covariate_omnibus_old.columns))]
    covariate_omnibus_old=covariate_omnibus_old.reset_index()
    if covariate_omnibus_old.shape[1]>1:
        covariate_omnibus_old[covariate_omnibus_old.columns[1:]].to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar.old'.format(idx),index=None,sep='\t',na_rep='NA')
        #covariate_omnibus[[covariate_omnibus.columns[0]]].to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar'.format(idx),index=None,sep='\t')
    """
    
    conditional_plink=conditional_plink_list+conditional_plink_phased_list
    dosage_array=np.array([np.where(plink.get_geno_marker(conditional_variant)==-1, np.nan, plink.get_geno_marker(conditional_variant)) for conditional_variant in conditional_plink])
    pd.Series(conditional_plink)[get_dupbool(dosage_array,axis=0)].to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)
    
    dosage_list=[]
    for conditional_variant in conditional_plink:
        doage=plink_aa.get_geno_marker(conditional_variant)
        dosage[dosage==-1]=np.nan
        dosage_list.append(dosage)
    dosage_list=np.array([])
    
    
    conditional_omnibus=cconditional_omnibus_list+conditional_plink_phased_list
    np.array([np.where(plink.get_geno_marker(conditional_variant)==-1, np.nan, plink.get_geno_marker(conditional_variant)) for conditional_variant in conditional_plink_list+conditional_plink_phased_list])
    
    dosage_list=[]
    for conditional_variant in conditional_plink_list+conditional_plink_phased_list:
        print(conditional_variant)
        doage=plink.get_geno_marker(conditional_variant)
        dosage[dosage==-1]=np.nan
        dosage_list.append(dosage)
    dosage_list=np.array(dosage_list) 


    dupcol_find(dosage_list.transpose())

    
    

    
    
    
    pd.Series(conditional_plink_list+conditional_plink_phased_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)
    #pd.Series(conditional_omnibus_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx))
    
    pd.Series(conditional_omnibus_list+conditional_plink_phased_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx),index=None,sep='\t',header=False)
    
    #Run plink association test
    log.info("#########################################  STEP {}-2 Plink Association  #########################################".format(idx))
    command='plink2 --bfile {bfile} {assoc_mode} --pheno {pheno} {covar_on} {covar} --condition-list {condition_list} --out {out} --covar-variance-standardize --threads 40'.format(bfile=plink_path,
                                                                                                                        assoc_mode='--logistic' if phenotype_type=='binary' else '--linear',
                                                                                                                         pheno=data_out_pheno_path+phenotype_name+'.phe',
                                                                                                                         covar_on='--covar' if len(covariate_plink.columns[1:])!=0 else '',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx) if len(covariate_plink.columns[1:])!=0 else '',
                                                                                                                         condition_list=data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),
                                                                                                                         out=data_out_assoc_phenotype_path+'step_{:02d}.plink'.format(idx)                                                                         
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)
    

    
    log.info("#########################################  STEP {}-3 Omnibus Association  #########################################".format(idx))
    #Run omnibus association test
    command='python omnibus_test.py --assoc {assoc_mode} --fam {fam} --aa {aa} --pheno {pheno} {covar_on} {covar} --condition-list {condition_list} --out {out}'.format(assoc_mode='logistic' if phenotype_type=='binary' else 'linear',
                                                                                                                         aa=data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx),
                                                                                                                         fam=plink_path+'.fam',
                                                                                                                         pheno=data_out_pheno_path+phenotype_name+'.phe',
                                                                                                                         covar_on='--covar' if len(covariate_omnibus.columns[1:])!=0 else '',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar'.format(idx) if len(covariate_omnibus.columns[1:])!=0 else '',
                                                                                                                         condition_list=data_out_assoc_phenotype_path+'step_{:02d}.omnibus.cond'.format(idx),
                                                                                                                         out=data_out_assoc_phenotype_path+'step_{:02d}.omnibus'.format(idx)
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)    
    
    """
    log.info("#########################################  STEP {}-3 Omnibus Association  #########################################".format(idx))
    #Run omnibus association test
    command='Rscript OmnibusTest_BHv5_modified.R {assoc_mode} {output} {fam} {aa} {phe} {covar} {covarname} {condvar}'.format(assoc_mode='logistic' if phenotype_type=='binary' else 'linear',
                                                                                                                         output=data_out_assoc_phenotype_path+'step_{:02d}.omnibus'.format(idx),
                                                                                                                         aa=data_out_assoc_phenotype_path+'step_{:02d}.aa'.format(idx),
                                                                                                                         fam=plink_path+'.fam',
                                                                                                                         phe=data_out_assoc_phenotype_path+'phenotype.phe',
                                                                                                                         covar=data_out_assoc_phenotype_path+'step_{:02d}.omnibus.covar.old'.format(idx) if len(covariate_omnibus.columns[1:])!=0 else 'NA',
                                                                                                                         covarname=','.join(covariate_omnibus_old.columns[1:]) if len(covariate_omnibus_old.columns[1:])!=0 else 'NA',
                                                                                                                         condvar=','.join(conditional_omnibus_list+conditional_plink_phased_list) if len(conditional_omnibus_list+conditional_plink_phased_list)!=0 else 'NA'
                                                                                                                        )
    log.info(command)
    stdout,stderr=run_subprocess(command,dry=False)
    log.info(stdout)
    log.error(stderr)
    
    """
    
    log.info("#########################################  STEP {}-4 Result Save  #########################################".format(idx))
    
    #Data save
    #plink_assoc_result=parse_plink_assoc(data_out_assoc_phenotype_path+'step_{:02d}.plink.{}assoc'.format(idx,'' if phenotype_type=='binary' else 'q'))
    #plink_assoc_result.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.{}assoc.munged'.format(idx,'' if phenotype_type=='binary' else 'q'))
    #plink_assoc_result_sorted=plink_assoc_result.sort_values('P')    
    #plink_assoc_result_sorted.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.{}assoc.sorted_'.format(idx,'' if phenotype_type=='binary' else 'q'),sep='\t',index=None)    
    
    plink_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t').astype({'P':float})
    plink_assoc_result=plink_assoc_result[plink_assoc_result['TEST']=='ADD']
    plink_assoc_result.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}.munged'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    plink_assoc_result_sorted=plink_assoc_result.sort_values('P')    
    plink_assoc_result_sorted.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}.sorted'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    
    omnibus_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.{}'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t').astype({'P':float})
    omnibus_assoc_result.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.{}.munged'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    omnibus_assoc_result_sorted=omnibus_assoc_result.sort_values('P')
    omnibus_assoc_result_sorted.to_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.{}.sorted'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t',index=None)
    
    #Identify Peak variant
    log.info("Plink association result")
    log.info(plink_assoc_result_sorted.head(5))
    log.info("Omnibus association result")
    log.info(omnibus_assoc_result_sorted.head(5))
    
    if plink_assoc_result_sorted.iloc[0]['P'] < omnibus_assoc_result_sorted.iloc[0]['P']:
        variant_name=plink_assoc_result_sorted.iloc[0]['ID']
        log.info('Plink win!!! {}'.format(variant_name))
        conditional_variant_list.append(("plink",variant_name))        
        if plink_assoc_result_sorted.iloc[0]['P']>5e-8:
            log.info('p-value of {}({}) is larger than 5x10^-8. finish conditional analysis'.format(variant_name,plink_assoc_result_sorted.iloc[0]['P']))
            break
    else:
        variant_name=omnibus_assoc_result_sorted.iloc[0]['ID']
        log.info('Omnibus win!!! {}'.format(variant_name))
        conditional_variant_list.append(("omnibus",variant_name))
        if omnibus_assoc_result_sorted.iloc[0]['P']>5e-8:
            log.info('p-value of {}({}) is larger than 5x10^-8. finish conditional analysis'.format(variant_name,omnibus_assoc_result_sorted.iloc[0]['P']))
            break
    

In [ ]:
pd.Series(conditional_plink_list+conditional_plink_phased_list).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)

In [ ]:
pd.Series(conditional_plink_list+conditional_plink_phased_list)[dupcol_find(dosage_list.transpose())].to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)

In [ ]:
get_dupbool??

In [ ]:
def get_dupbool(array,axis=1,threshold=0.9,keep='last'):
    array=array.copy()
    if isinstance(array, np.ndarray):
        pass
    elif isinstance(array, pd.core.frame.DataFrame):
        array=array.values
        
    if axis==1:
        pass
    elif axis==0:
        array=array.transpose()
    else:
        raise NotImplementedError
            
    dupcol_check=np.repeat(False,array.shape[1])
    for i in range(array.shape[1]):
        for j in range(i+1,array.shape[1]):
            x_i=array[:,i]
            x_j=array[:,j]
            if i==8 or j==8:
                print(i,j,np.corrcoef(x_i,x_j)[0,1])
            if np.all(x_i==x_j) or np.abs(np.corrcoef(x_i,x_j)[0,1])>threshold:
                if keep==False:
                    dupcol_check[i]=True
                    dupcol_check[j]=True
                elif keep=='last':   
                    if (not dupcol_check[i]) and (not dupcol_check[j]):
                        dupcol_check[j]=True
                elif keep=='loose_last':
                    dupcol_check[j]=True
                else:
                    raise NotImplementedError
    return dupcol_check

In [ ]:
conditional_plink=conditional_plink_list+conditional_plink_phased_list
pd.Series(conditional_plink)[~get_dupbool(np.array([plink.get_geno_marker(conditional_variant) for conditional_variant in conditional_plink]),axis=0,threshold=0.0001,keep=False)].to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(idx),index=None,sep='\t',header=False)


In [ ]:
[~get_dupbool(np.array([plink.get_geno_marker(conditional_variant) for conditional_variant in conditional_plink]),axis=0,threshold=0.7,keep='last')]

In [ ]:
[~get_dupbool(np.array([plink.get_geno_marker(conditional_variant) for conditional_variant in conditional_plink]),axis=0,threshold=0.000001,keep=False)]

In [ ]:
!plink2 --bfile data/genotype/4_merge/KCHIP_HLA_SNP_1000G_merged --linear --pheno data/out_pheno/ALP.phe --covar data/out_assoc/ALP/step_05.plink.covar --condition-list data/out_assoc/ALP/step_05.plink.cond --out data/out_assoc/ALP/step_05.plink --covar-variance-standardize --threads 40

In [ ]:
covariate_plink=covariate_plink.drop(columns=covariate_plink.columns[1:][get_dupbool(covariate_plink.iloc[:,1:],threshold=0.5)])

In [ ]:
if covariate_plink.shape[1]>1:
    covariate_plink[[covariate_plink.columns[0],covariate_plink.columns[0]]+covariate_plink.columns[1:].values.tolist()].fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')

In [ ]:
covariate_plink.iloc[:,1:][:,]

In [ ]:
get_dupbool(covariate_plink.iloc[:,1:],threshold=0.7)

In [ ]:
covariate_plink.iloc[:,1:].columns

In [ ]:
if covariate_plink.shape[1]>1:
    covariate_plink[[covariate_plink.columns[0],covariate_plink.columns[0]]+covariate_plink.columns[1:].values.tolist()].fillna(-9).to_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(idx),index=None,sep='\t')

In [ ]:
get_dupbool(covariate_plink.iloc[:,1:],threshold=0.7,keep=False)

In [ ]:
covariate_plink.shape

In [ ]:
dupcol_list=dupcol_find(covariate_plink.iloc[:,1:])

In [ ]:
dupcol_list=dupcol_find(covariate_plink.iloc[:,1:])

covariate_plink=covariate_plink.drop(columns=dupcol_list)


In [ ]:
for i in range(1,len(covariate_plink.columns)):
    print(covariate_plink.columns[i],covariate_plink.columns[i+1])
    print(np.unique(covariate_plink.iloc[:,i]+covariate_plink.iloc[:,i+1]))

In [ ]:

dupcol_find(covariate_plink.iloc[:,1:])    

In [ ]:
#np.linalg.matrix_rank(x_data_covar)

In [ ]:
corr=covariate_plink.corr()

In [ ]:
np.linalg.matrix_rank(corr.values[:,:])

In [ ]:
covariate_plink.shape

In [ ]:
a=np.array([plink.get_geno_marker(conditional_variant) for conditional_variant in conditional_plink_list+conditional_plink_phased_list])

In [ ]:
get_dupbool(a,axis=0,threshold=0.9,keep=False)

In [ ]:
idx

In [ ]:
a.shape

In [ ]:
[plink.get_geno_marker(conditional_variant) for conditional_variant in (conditional_plink_list+conditional_plink_phased_list)]

In [ ]:
dosage_list=[]
for conditional_variant in conditional_plink_list+conditional_plink_phased_list:
    #print(conditional_variant)
    dosage=plink.get_geno_marker(conditional_variant)
    #dosage[dosage==-1]=np.nan
    dosage_list.append(dosage)
dosage_list

In [ ]:
dosage_list=np.array(dosage_list) 
dosage_list.shape

In [ ]:
def dupcol_find(array,axis=1,threshold=0.99):
    array=array.copy()
    if isinstance(array, np.ndarray):
        pass
    elif isinstance(array, pd.core.frame.DataFrame):
        array=array.values
        
    if axis==1:
        pass
    elif axis==0:
        array=array.transpose()
    else:
        raise NotImplementedError
            
    dupcol_check=np.repeat(False,array.shape[1])
    for i in range(array.shape[1]):
        for j in range(i+1,array.shape[1]):
            if (not dupcol_check[i]) and (not dupcol_check[j]):
                x_i=array[:,i]
                x_j=array[:,j]
                if np.all(x_i==x_j) or np.corrcoef(x_i,x_j)[0,1]>threshold:
                    dupcol_check[j]=True
    return dupcol_check